## 付録E　電気ヒートポンプ給湯機

### E.1 記号及び単位

省略

### E.2 消費電力量

1日当たりの沸き上げ時間$ \Delta t_{wh} $は 8 時間とする。

$$\large
E_{E,hs,d,t} = \left\{ \begin{array}{ll}
    (f_{ex} \times \theta_{ex,NAve,d} + f_{L} \times L''_{d} + f_{e,rtd} \times e_{rtd} + f_{itc})\\
    \large \times (1 + f_{mode2} \times r_{mode2}) \times C_{LT,d} \times C_{def,d} \times 10^{3} \div 3600 \div \Delta t_{wh} & \big(t=0～6 \big)\\ 
    0 & \big(t=7～22 \big)\\ 
    (f_{ex} \times \theta_{ex,NAve,d+1} + f_{L} \times L''_{d+1} + f_{e,rtd} \times e_{rtd} + f_{itc})\\
    \large \times (1 + f_{mode2} \times r_{mode2}) \times C_{LT,d+1} \times C_{def,d+1} \times 10^{3} \div 3600 \div \Delta t_{wh} & \big(t=23 \big)\\ 
    \end{array}
\right.
$$

$C_{def,d}$：日付$d$における1 日当たりのデフロスト運転による補正係数(-)  
$C_{LT,d}$：日付$d$における1 日当たりの低外気温領域でのヒートポンプ効率低下による補正係数(-)  
$E_{E,hs,d,t}$：日付$d$の時刻$t$における1 日当たりの給湯機の消費電力量(kWh/h)  
$e_{rtd}$：当該給湯機の効率(-)  
$L''_{d}$：日付$d$における1 日当たりの太陽熱補正給湯熱負荷(MJ/d)  
$r_{mode2}$：制御におけるセカンドモードの割合(-)  
$\Delta t_{wh}$：1日当たりの沸き上げ時間(h/d)  
$\theta_{ex,NAve,d}$：日付$d$における夜間平均外気温度(℃)  
$f_{ex}, f_{L}, f_{e,rtd}, f_{itc}, f_{mode2}$：係数(-)  


<div style="text-align: center;font-weight: bold;">表　係数</div>

<table>
<tr><th>$f_{ex}$</th><td>-0.302</td></tr>
<tr><th>$f_{L}$</th><td>0.263</td></tr>
<tr><th>$f_{e,rtd}$</th><td>-4.383</td></tr>
<tr><th>$f_{itc}$</th><td>20.754</td></tr>
<tr><th>$f_{mode2}$</th><td>0.235</td></tr>
</table>


ふろ機能の種類が「ふろ給湯機（追焚あり）」の場合

$$\large
e_{rtd} = e_{APF} -  0.7 
$$

ふろ機能の種類が「給湯単機能」又は「ふろ給湯機（追焚なし）」の場合

$$\large
e_{rtd} = e_{APF} -  0.5 
$$

$e_{APF}$：年間給湯効率(-)  
$e_{rtd}$：当該給湯機の効率(-)  

換算した値は、3.6 を超える場合には 3.6 に等しいとする。

制御におけるセカンドモードの割合$r_{mode2}$は0.4とする。 

$$\large
C_{LT,d} = \left\{ \begin{array}{ll}
    1.0 &(7 \leq \theta_{ex,Nave,d}の場合)\\ 
    1.0 + 0.0133 \times (7 - \theta_{ex,Nave,d}) &(\theta_{ex,Nave,d} < 7の場合)\\ 
    \end{array}
\right.
$$

$C_{LT,d}$：日付$d$における1 日当たりの低外気温領域でのヒートポンプ効率低下による補正係数(-)  
$\theta_{ex,NAve,d}$：日付$d$における夜間平均外気温度(℃)  

$$\large
C_{def,d} = \left\{ \begin{array}{ll}
    1.0 &(5 \leq \theta_{ex,Nave,d}の場合)\\ 
    1.175 - \theta_{ex,Nave,d} \times 0.035 &(2 \leq \theta_{ex,Nave,d}<5の場合)\\ 
    1.12 - \theta_{ex,Nave,d} \times 0.0075 &(-2 \leq \theta_{ex,Nave,d}<2の場合)\\ 
    1.155 + \theta_{ex,Nave,d} \times 0.01 &(-15.5 \leq \theta_{ex,Nave,d}<-2の場合)\\ 
    1.0 &(\theta_{ex,Nave,d}<-15.5の場合)\\ 
    \end{array}
\right.
$$

$C_{def,d}$：日付$d$における1 日当たりのデフロスト運転による補正係数(-)  
$\theta_{ex,NAve,d}$：日付$d$における夜間平均外気温度(℃)  

### E.3 ガス消費量

日付$d$の時刻$t$における 1時間当たりの給湯機のガス消費量$E_{G,hs,d,t}$は0とする。

### E.4 灯油消費量

日付$d$の時刻$t$における 1時間当たりの給湯機の灯油消費量$E_{K,hs,d,t}$は0とする。

### E.5 １日当たりの太陽熱補正給湯熱負荷

$$\large
\displaystyle
L''_{d} = L''_{k,d}+L''_{s,d}+L''_{w,d}+L''_{b1,d}+L''_{b2,d}+L''_{ba1,d}+L''_{ba2,d}
$$

$ L''_{d} $：日付$d$における1 日当たりの太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{k,d} $：日付$d$における1 日当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{s,d} $：日付$d$における1 日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{w,d} $：日付$d$における1 日当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{b1,d} $：日付$d$における1 日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{b2,d} $：日付$d$における1 日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{ba1,d} $：日付$d$における1 日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{ba2,d} $：日付$d$における1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d)  

$$\large
L''_{k,d} = \sum_{t=0}^{23} L''_{k,d,t}
$$

$$\large
L''_{s,d} = \sum_{t=0}^{23} L''_{s,d,t}
$$

$$\large
L''_{w,d} = \sum_{t=0}^{23} L''_{w,d,t}
$$

$$\large
L''_{b1,d} = \sum_{t=0}^{23} L''_{b1,d,t}
$$

$$\large
L''_{b2,d} = \sum_{t=0}^{23} L''_{b2,d,t}
$$

$$\large
L''_{ba1,d} = \sum_{t=0}^{23} L''_{ba1,d,t}
$$

$$\large
L''_{ba2,d} = \sum_{t=0}^{23} L''_{ba2,d,t}
$$

$L''_{k,d}$：：日付$d$における1 日当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{s,d}$：：日付$d$における1 日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{w,d}$：：日付$d$における1 日当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{b1,d}$：：日付$d$における1 日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{b2,d}$：：日付$d$における1 日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{ba1,d}$：日付$d$における1 日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{ba2,d}$：日付$d$における1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{k,d,t} $：日付$d$の時刻$t$における 1時間当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{s,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{w,d,t} $：日付$d$の時刻$t$における 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b2,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{ba1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{ba2,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/h)  